In [1]:
import numpy as np
from datetime import datetime

from apsuite.commisslib.meas_bpms_signals import AcqBPMsSignals
from siriuspy.devices import BOPSRampStandbyHandler, BORFRampStandbyHandler, \
    BunchbyBunch, HLFOFB, SOFB


In [2]:
def configure_acquisition_params(orbacq, timeout=40, nrptsbefore=1000, nrptsafter=10000):
    """."""
    params = orbacq.params
    params.signals2acq = 'XY'
    params.acq_rate = 'FAcq' 
    params.timeout = 40

    params.nrpoints_before = nrptsbefore  # Default: 1000
    params.nrpoints_after = nrptsafter  # Default: 10000
    params.acq_repeat = False
    params.trigbpm_delay = None
    params.trigbpm_nrpulses = 1

    params.timing_event = 'Study' 
    params.event_mode = 'External'
    params.event_delay = None
    params.do_pulse_evg = False  # Default: False
    print('--- orbit acquisition configuration ---')
    print(params)


def measure(orbacq):
    """."""
    init_state = orbacq.get_timing_state()
    orbacq.prepare_timing()
    event_mode = orbacq.params.event_mode
    if event_mode.lower() == 'external':
        print((
            'Please check BPM acquisition status, or wait a few seconds, '
            ' before triggering event!'))
    else:
        print('Waiting for next injection pulse')
        print((
            'Take a look at Injection Control Log to check for next '
            'injection'))
    try:
        orbacq.acquire_data()
    except Exception as e:
        print(f"An error occurred during acquisition: {e}")
    # Restore initial timing state, regardless acquisition status
    orbacq.recover_timing_state(init_state)
    return orbacq.data is not None


def initialize(timeout, nrptsbefore, nrptsafter):
    """."""
    orbacq = AcqBPMsSignals(isonline=True)
    configure_acquisition_params(orbacq, timeout, nrptsbefore, nrptsafter)
    print('--- orbit acquisition connection ---')
    print(orbacq.wait_for_connection(timeout=100))
    return orbacq


def create_devices():
    """."""
    sofb = SOFB(SOFB.DEVICES.SI)
    fofb = HLFOFB(HLFOFB.DEVICES.SI)
    bbbl = BunchbyBunch(BunchbyBunch.DEVICES.L)
    bbbh = BunchbyBunch(BunchbyBunch.DEVICES.H)
    bbbv = BunchbyBunch(BunchbyBunch.DEVICES.V)
    bopsrmp = BOPSRampStandbyHandler()
    borfrmp = BORFRampStandbyHandler()
    return sofb, fofb, bbbl, bbbh, bbbv, bopsrmp, borfrmp


def read_feedback_status(devs, orbacq):
    """."""
    sofb, fofb, bbbl, bbbh, bbbv, bopsrmp, borfrmp = devs
    orbacq.data['sofb_loop_state'] = sofb.autocorrsts
    orbacq.data['fofb_loop_state'] = fofb.loop_state
    orbacq.data['bbbl_loop_state'] = bbbl.feedback.loop_state
    orbacq.data['bbbh_loop_state'] = bbbh.feedback.loop_state
    orbacq.data['bbbv_loop_state'] = bbbv.feedback.loop_state
    orbacq.data['bo_ps_ramp_state'] = bopsrmp.is_on
    orbacq.data['bo_rf_ramp_state'] = borfrmp.is_on

In [ ]:
timeout = 40
nrptsbefore=1000
nrptsafter=10000

orbacq = initialize(timeout, nrptsbefore, nrptsafter)
devs = create_devices()

if measure(orbacq):
    read_feedback_status(devs, orbacq)
    now = datetime.now()
    str_rate = f'FAcq_rate_'
    str_now = now.strftime('%Y-%m-%d_%Hh%Mm%Ss')
    filename = 'IVU18mov_' + str_rate + str_now
    orbacq.save_data(filename, overwrite=False)
    print(f'\nData saved at {str_now:s}')
else:
    print('\nData NOT saved!')